# Overview:
This notebook is mainly designed to save my notes on Knowledge distillation papers.
I will use this [survey](https://arxiv.org/pdf/2006.05525.pdf) as my guide to the Knowledge distillation world.

# Response-Based Knownledge
This type of KD is based solely on the logits: outputs of the last fully connected layer of the teached model.
It can generalized for most computer vision tasks: classification, segmentation, and object detection.

## Paper: Vanilla Knowledge Distillation 
* [link](https://arxiv.org/pdf/1503.02531.pdf)
* no code link in the paper

* main ideas:
    1. The main idea is a random variable with high entropy is quite informative. In other words, if we have an instance $x_i$, associated with probabilities $p_A = 10 ^ {-6}$, $p_B = 8 * 10 ^ {-5}$ and $p_C = 10 ^ {-2}$, if similar distributions are observed across a large number of instances we can say that $A$ and $B$ are quite similar. 
    2. Basically, soft labels incorporate much more knowledge than hard labels.
    3. Cumbersome model (CM) and distilled Model (DM). Train the model on some large training data. leave a holdout dataset generally referred to as the ***transfer dataset***. Minimize entropy between the logits (probabilities are too low to affect the entropy loss) of CM and DM
    4. Having some of transfer data labeled (or all of it) with CM's predictions. The loss function can be updated by adding a term where the DM should predict correctly.
    5. The mathematical details are not shared, so I can't tell for sure how the process work.
    6. The softmax function is defined as: $$p_i = \frac{exp(\frac{z_i}{T})}{\sum_j exp(\frac{z_j}{T})} $$
    7. The value $T$ is referred to as the temperature. The larger the value of $T$, the softer, more uniform the distribution will get.
    8. if DM is relatively large, high values of $T$ should be used. For relatively smaller DMs (less neuron units, less layers), use smaller $T$
    9. $T$ is used in the softmax predictions of the DM model on the transfer dataset


* funny enough (I am still not used to how research papers work), this [FitNet](https://arxiv.org/pdf/1412.6550.pdf) explains the vanilla Knowledge Distillation:
    1. So when the labels are not known, the student is trained to minimize $$ L(P^{\tau}_{T}, P^{\tau}_{S})$$ where $P^{\tau}$ is the output of the softmax with temperature $T$
    2. when some of the labeled data is known, the loss can be modified as follows: $$ L(y_{true}, P^{\tau}_{S}) + \lambda \cdot L(P^{\tau}_{T}, P^{\tau}_{S})$$


The paper share a number of ideas concerning distilling ensemble of models, but it does not serve the topic of my research (at least I don't see how, yet).
Reading Smooth labeling is a must.

* IMPORTANT NOTE: the temperature $T$ is used only in the training phase. The usual softmax with temperature $1$ is used in inference. 

* There is another similar [paper](https://arxiv.org/pdf/1312.6184.pdf). Its author experiemented mainly with $L_2$ loss as well as introduce a small matrix factorization trick to speed up the training process.

* There are other significant distributions to the response based KD literature that should be considered further:
* [CLASS-DISTANCE LOSS](https://openreview.net/pdf?id=ByXrfaGFe)
* [Label Smoothing](https://arxiv.org/abs/1906.02629)
* [Adaptive Regularization of labels](https://arxiv.org/abs/1908.05474)


# Feature Based Knowledge Distillation
This is the 2nd category of Knowledge distillation, which can also be seen as deep Knowledge distillation where the process uses the outputs of intermediate layers.

## FiTNets: hint-based Knonwledge Destillation
* [link](https://arxiv.org/pdf/1412.6550.pdf)
* [code](https://github.com/adri-romsor/FitNets)

* The main idea:
    1. Using the hidden layers of the teacher model in the training
    2. Training the student model progressively, by training a selection of layers (they should be in the beginning of the network as later layers are more restrective
    and have less transferable knowledge)
    3. optimize each layer by optimizing the loss function: $L(W_{guided}, W_{r}) = \frac{1}{2} \cdot ||u_{t}(x, W_{hint}) - r\cdot(u_{s}(x, W_{guided}), W_r)||$  
    where $u_{t}$ is the output of the teacher's model up to that layer (with parameters $W_{hint}$). and $u_{s}$ is the student model output up to the guided layer. 
    4. it is important to keep in mind that the shape of the guided layer output  might be different from the one with the hint layer. Thus, the output should be modified with the regressor (with parameters $W_r$).
    5. The regressor can be chosen as a convolution neural network for efficienty (instead of a fully connected network which is likely to introduce signficant overhead)
    6. after assigning the new parameters for the different hidden layers in the student model, the student model is further trained with the Vanillay distillation loss function.
![algorithm's summary](../images/FitsNets_algo.png)



## Exclusivity-Consistency Regularized KD
* [paper](https://www.ecva.net/papers/eccv_2020/papers_ECCV/papers/123690324.pdf)
* [code](http://www.cbsr.ia.ac.cn/users/xiaobowang/)

* KD is the most promising model compression approach as it enables knowledge transfer regardless of the architectural differences between teacher and student models.
* The authors of this paper focus on an important idea. If the model is low capacity, the parameters should be as diverse as possible.
* A single filter in a convolutional layer is of the shape: $(M, K_1, K_2)$ where $M$ is the number of input channels, $K_1, K_2$ are the kernel's dimensions: We can flatten all the parameters into a single vector of length $D = M \cdot K_1 \cdot K_2$ and the entire convolutional layer can be represented as 2-dimensional matrix $W = \begin{bmatrix} w_1 & w_2 & .. & w_N\end{bmatrix}^T$ where $w_i$ is the flattened vector of the $i-th$ filter.
* The filter of the student layers should be as diverse as possible, minimizing their similarity in a sense:
        $$L_{WE} ||w_i, w_j||_1 = \sum_{1\leq i < j \leq N} |w_i|  \bigodot |w_j|$$ 
where ***WE*** stands for Weight Exclusivity.

* They additionally tweak feature consistency by considering the features of the last layer and emphasizing samples with significantly different representation (features). The the authors mathematically model this idea in several steps as follows:
1. $L_{FC} = H||F_S - F_T||$ where $H$ is the $L_2$ loss.
2. each sample is a mini-batch is associated a weight: $s_i = \frac{e ^ {H_j}}{\sum_i^m e^{H_i}}$ 
3. The Feature consistency loss is as follows: $L_{HFC} = (1 + s_i) \cdot H(F_s, F_t)$

* The final distillation loss: $L = L_{HFC} + \lambda_1 \cdot ||W||^2 + \lambda_2 \cdot L_{WE}$

* This paper contains additional important ideas in the evaluation part: evaluating the proposed methods at tackling certain parts.

## Route Constrained Optimization

* [paper](https://openaccess.thecvf.com/content_ICCV_2019/papers/Jin_Knowledge_Distillation_via_Route_Constrained_Optimization_ICCV_2019_paper.pdf)
* no code 

* The main idea introduced by this paper is incorporating the training process of the teacher model in the knownledge distillation process.
* The local minima reached be the teacher model might be difficult to achieve by the student model due to the capacity gap. 
* Thus guiding the student towards the same trajectory taken by the teacher model in the search space can achieve similar if not superior results.
* Assuming we have $n$ anchor points (or checkpoints...) $C_1, C_2, ... C_n$ where each checkpoint represents the logits of the teacher model, the KD algorithm is as follows:
![algorithm](../images/rco_algo.png)

* technically the traning of the student network is split in $n$ stages where the student model tries to reach the $n$-th point in the model's learning route.
* The choice of anchors points might be the method's reason to succeed or fail. The paper suggests several strategies. 

## Neuron selectivity Transfer
* [paper](https://arxiv.org/pdf/1707.01219.pdf)
* The paper perceives knowledge from a neuron activation perspective. In other words, similar images (let's focus on classes at the very least) should activate the similar neurons (With similar distributions). Now, the idea is as follows: Assuming we have the teacher and student CNNs, $T$ and $S$ with potenitally different dimensions and with the following outputs f with dimensions $C \cdot N \cdot H$ where $C$ is the number of filters.:
* the authors use an advanced distribution distance metric, MMD with kernel tricks. Aligning the distributions aligh the neuron activity and thus the knowledge.
* additional reading: Kernels and Kernel tricks.

# Relation-Based Knowledge Distillation
* Certain researchers suggest that using the output of specific layers might present a constraint on the student's network learning. Relation based is a slightly different directions where the student model does not learn to mimic the teacher model's outputs but structural / architectural features.
## A Gift from Knowledge distillation
* [link](https://openaccess.thecvf.com/content_cvpr_2017/papers/Yim_A_Gift_From_CVPR_2017_paper.pdf)
* The authors represent knowledge as the ***'texture for the feature maps'***. Assuming a feature map $FM_1$ of shape $(h, w, n)$ and another feature map $FM_2$ of shape $(h, w, n)$ then the authors define this knowledge as $$G_{i, j} = \sum_{s=1}^h \sum_{t=1}^w \frac{FM_1(s, t, i) \cdot FM_2(s, t, j)}{h \cdot w}$$
* technically, taking 2 feature maps with the same kernel size (with possibly different numbers of channels), the texture of these 2 feature maps (also referred to as the FSP matrices) is defined as the sum of the element wise product between any pair of filters in the feature maps.

* choosing feature maps with the same kernel size from the student and teacher networks, then the loss is defined as
        $$ \frac{1}{m} \sum_x \sum_{i=1}^n \lambda_i \cdot || G_i(x, W_T) - G_i(x, W_S)||_2$$
where $m$ is the number of samples in whatever batch, we are considering.
